In [1]:
import os
import math
import numpy as np
import pandas as pd

In [2]:
data_path = 'data/'

### Dataset Movie Lens

In [3]:
users = pd.read_csv(
    data_path + "ml-1m/users.dat",
    sep="::",
    names=["user_id", "sex", "age_group", "occupation", "zip_code"],
    engine="python",
)

ratings = pd.read_csv(
    data_path + "ml-1m/ratings.dat",
    sep="::",
    names=["user_id", "movie_id", "rating", "unix_timestamp"],
    engine="python",
)

movies = pd.read_csv(
    data_path + "ml-1m/movies.dat",
    sep="::",
    names=["movie_id", "title", "genres"],
    engine="python",
    encoding="latin-1",
)

In [4]:
users.head()

,user_id,sex,age_group,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [28]:
ratings.head()

,user_id,movie_id,rating
0,user_1,movie_1193,5.0
1,user_1,movie_661,3.0
2,user_1,movie_914,3.0
3,user_1,movie_3408,4.0
4,user_1,movie_2355,5.0


In [22]:
movies.head()

,movie_id,title,genres,year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama,1995
4,5,Father of the Bride Part II,Comedy,1995


In [16]:
mv = movies.copy()

In [21]:
movies = mv.copy()

In [26]:
users["user_id"] = users["user_id"].apply(lambda x: f"user_{x}")
users["age_group"] = users["age_group"].apply(lambda x: f"group_{x}")
users["occupation"] = users["occupation"].apply(lambda x: f"occupation_{x}")

movies["movie_id"] = movies["movie_id"].apply(lambda x: f"movie_{x}")
movies["year"] = movies["title"].apply(lambda x : int(x[-5:-1]))
movies["title"] = movies["title"].apply(lambda x : x[:-7].to_lower)

ratings["movie_id"] = ratings["movie_id"].apply(lambda x: f"movie_{x}")
ratings["user_id"] = ratings["user_id"].apply(lambda x: f"user_{x}")
ratings["rating"] = ratings["rating"].apply(lambda x: float(x))

ratings.drop(columns="unix_timestamp", inplace=True)

,user_id,movie_id,rating
0,user_1,movie_1193,5.0
1,user_1,movie_661,3.0
2,user_1,movie_914,3.0
3,user_1,movie_3408,4.0
4,user_1,movie_2355,5.0
...,...,...,...
1000204,user_6040,movie_1091,1.0
1000205,user_6040,movie_1094,5.0
1000206,user_6040,movie_562,5.0
1000207,user_6040,movie_1096,4.0


In [47]:
len(movies)

3883

In [46]:
### pas besoin y en a pas
movies.drop_duplicates(inplace=True)

### dataset imdb

In [5]:
# name_basics = pd.read_csv(
#     data_path + r"imdb/name.basics.tsv",
#     sep='\t',
#     engine="python",
#     # low_memory=False
# )

title_basics = pd.read_csv(
    data_path + r"imdb/title.basics.tsv",
    sep='\t',
    # engine="python",
    low_memory=False
)

title_ratings = pd.read_csv(
    data_path + r"imdb/title.ratings.tsv",
    sep='\t',
    # engine="python",
    low_memory=False
)


In [29]:
title_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [44]:
title_basics[title_basics["primaryTitle"]  != title_basics["originalTitle"]]
### primary title = nom anglais donc plus de match avec movie lens
             

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895,\N,1,"Documentary,Short"
11,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0,1896,\N,1,"Documentary,Short"
12,tt0000013,short,The Photographical Congress Arrives in Lyon,Le débarquement du congrès de photographie à Lyon,0,1895,\N,1,"Documentary,Short"
13,tt0000014,short,The Waterer Watered,L'arroseur arrosé,0,1895,\N,1,"Comedy,Short"
15,tt0000016,short,Boat Leaving the Port,Barque sortant du port,0,1895,\N,1,"Documentary,Short"
...,...,...,...,...,...,...,...,...,...
10855747,tt9916124,movie,The Taste Is Mine,Mucho gusto,0,1997,\N,108,Documentary
10855768,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,\N,51,Drama
10855861,tt9916362,movie,Coven,Akelarre,0,2020,\N,92,"Drama,History"
10855893,tt9916428,movie,The Secret of China,Hong xing zhao yao Zhong guo,0,2019,\N,\N,"Adventure,History,War"


In [33]:
unique_type = title_basics['titleType'].unique()
print("Valeurs différentes de la colonne 'title Type':", unique_type)

Valeurs différentes de la colonne 'title Type': ['short' 'movie' 'tvShort' 'tvMovie' 'tvSeries' 'tvEpisode' 'tvMiniSeries'
 'tvSpecial' 'video' 'videoGame' 'tvPilot']


In [42]:
movietype = ["movie", "tvMovie"]
title_preprocess = title_basics[title_basics["titleType"].isin(movietype)].copy()
title_preprocess['titleType'].unique()

array(['movie', 'tvMovie'], dtype=object)

In [43]:
### on drope endYear car la colonne ne fait sens que pour les TV series
title_preprocess.drop(columns="endYear", inplace=True)